In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import combine_svm
import pandas as pd
from tqdm import tqdm
import scipy.signal as signal
import csv
import seaborn as sns
sns.set()


In [3]:
def feature_detection(smoothing_signal):
    feature_peak, _ = signal.find_peaks(smoothing_signal)
    feature_valley, _ = signal.find_peaks(-smoothing_signal)
    data_value = np.multiply(np.square(smoothing_signal), np.sign(smoothing_signal))
    return feature_peak, feature_valley, data_value

def caculate_breathrate(NT_points, NB_points):
    # if both NT and NB are not detected
    if NT_points.shape[0] <= 1 and NB_points.shape[0] <= 1:
        return None
    # if only NT are detected
    elif NT_points.shape[0] > 1 and NB_points.shape[0] <= 1:
        tmp = np.concatenate(([0], NT_points), axis=0)
        tmp_2 = np.concatenate((NT_points, [0]), axis=0)
        aver_NT = tmp_2[1:-1] - tmp[1:-1]
        return 1200 / np.mean(aver_NT)  # (60)*(20)
    # if only NB are detected
    elif NB_points.shape[0] > 1 >= NT_points.shape[0]:
        tmp = np.concatenate(([0], NB_points), axis=0)
        tmp_2 = np.concatenate((NB_points, [0]), axis=0)
        aver_NB = tmp_2[1:-1] - tmp[1:-1]
        return 1200 / np.mean(aver_NB)
    else:
        tmp = np.concatenate(([0], NT_points), axis=0)  # tmp 兩點距離
        tmp_2 = np.concatenate((NT_points, [0]), axis=0)
        aver_NT = tmp_2[1:-1] - tmp[1:-1]
        tmp = np.concatenate(([0], NB_points), axis=0)
        tmp_2 = np.concatenate((NB_points, [0]), axis=0)
        aver_NB = tmp_2[1:-1] - tmp[1:-1]
        aver = (np.mean(aver_NB) + np.mean(aver_NT)) / 2
    return 1200 / aver

''' ----------------------- Movement ----------------------- '''
# 30s訊號 = 30 * 20 = 600
def mov_dens_fn(raw_sig):
    new_sig = []
    count = 0
    for num in range(80):
        top = 0
        first = int(num*(0.5*20))
        last = int((num+1)*(0.5*20))
        x = raw_sig[first:last]

        # 方差公式
        for i in range(10):
            top += np.square(x[i] - np.average(x))
        result = top / (10 - 1)
        if result > 0.005:  # 閥值可調
            count += 1
    percent = (count/80) * 100
    return percent

''' ----------------------- Respiration ----------------------- '''
# fRSA (40sec)
def fRSA_fn(br_sig):
    auto_br_sig = np.correlate(br_sig, br_sig, mode='full')
    auto_br_sig = auto_br_sig[auto_br_sig.size//2:]
    feature_peak, feature_valley, _= feature_detection(auto_br_sig)
    fRSA = caculate_breathrate(feature_peak, feature_valley)
    fRSA = (fRSA / 2)
    return fRSA

# 10個fRSA
def tfRSA_fn(fRSA_sig):
    tfRSA = np.std(fRSA_sig)
    return tfRSA

# 31個f𝑅𝑆𝐴
def sfRSA_fn(fRSA_sig):
    sfRSA = scipy.signal.savgol_filter(fRSA_sig, 31, 3)
    sfRSA_mean = np.average(sfRSA)
    return sfRSA, sfRSA_mean

# 31個tf𝑅𝑆𝐴
def stfRSA_fn(tfRSA_sig):
    stfRSA = scipy.signal.savgol_filter(np.array(tfRSA_sig), 31, 2)
    stfRSA_mean = np.average(stfRSA)
    return stfRSA, stfRSA_mean

# 31個f𝑅𝑆𝐴
def sdfRSA_fn(fRSA, sfRSA):
    sdfRSA = np.abs(fRSA - sfRSA)
    sdfRSA = scipy.signal.savgol_filter(sdfRSA, 31, 3)
    sdfRSA_mean = np.average(sdfRSA)
    return sdfRSA, sdfRSA_mean

''' ----------------------- Heart rate ----------------------- '''
def mHR_fn(heart):
    return fRSA_fn(heart)

# 待補
def LF_HF_LFHF(sig):
    LF_sig = combine_svm.iir_bandpass_filter_1(amp_sig, 0.04, 0.15, 20, 9, "cheby2")
    HF_sig = combine_svm.iir_bandpass_filter_1(amp_sig, 0.15, 0.4, 20, 9, "cheby2")

def tmHR_fn(mHR_sig):
    return tfRSA_fn(mHR_sig)

def smHR_fn(mHR_sig):
    return sfRSA_fn(mHR_sig)

def stmHR_fn(tmHR_sig):
    return stfRSA_fn(tmHR_sig)

def sdmHR_fn(mHR, smHR):
    return sdfRSA_fn(mHR, smHR)


# 待補
def sHF_fn(HF_sig):
    sHF = scipy.signal.savgol_filter(HF_sig, 31, 3)
    sHF_mean = np.average(sfRSA)
    return sHF, sHF_mean

def sLFHF_fn(LFHF_sig):
    sLFHF = scipy.signal.savgol_filter(LFHF_sig, 31, 3)
    sLFHF_mean = np.average(sfRSA)
    return sLFHF, sLFHF_mean

In [37]:
names = "./dataset_sleep"
for name in os.listdir(names):
    files = os.path.join(names, name, "0.8")
    for num in range(len(os.listdir(files)) // 2):
        datas = os.listdir(files)[num]
        data = os.path.join(files, datas)
        raw_data = pd.read_csv(data)
        raw_data_pd = pd.DataFrame(raw_data)

In [29]:
pd.info(raw_data_pd.head)

AttributeError: module 'pandas' has no attribute 'info'

In [39]:
raw_data_pd['heart'] = raw_data_pd['heart'].replace(0, np.nan)
new_data = raw_data_pd.dropna()
new_data

,rangeBinIndexMax,rangeBinIndexPhase,maxVal,processingCyclesOut,processingCyclesOut1,rangeBinStartIndex,rangeBinEndIndex,unwrapPhasePeak_mm,outputFilterBreathOut,outputFilterHeartOut,...,rsv[3],rsv[4],rsv[5],rsv[6],rsv[7],rsv[8],rsv[9],datetime,heart,breath
800,0,24,177860256.0,9240,0,11,73,2.571598,-0.232901,-0.051293,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:24:29,85.0,17
820,0,24,109784712.0,9239,0,11,73,15.444456,0.466879,-1.299305,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:24:30,84.0,17
841,0,24,62075720.0,9246,0,11,73,22.603590,0.110401,-0.256688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:24:31,83.0,16
861,0,24,51116016.0,9247,0,11,73,16.592232,-0.477896,0.778677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:24:32,82.0,16
882,0,29,74466840.0,9245,0,11,73,17.494726,-0.068167,-0.082371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:24:33,81.0,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549365,0,31,137685104.0,9249,0,11,73,155.858994,-0.005111,-0.010837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08:01:38,81.0,15
549386,0,31,127933568.0,9237,0,11,73,155.870041,-0.002352,-0.001091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08:01:39,77.0,15
549406,0,31,141781280.0,9236,0,11,73,155.875443,-0.000543,-0.002475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08:01:40,72.0,15
549426,0,31,132085728.0,9239,0,11,73,155.883896,0.000375,-0.005654,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08:01:41,76.0,15


In [42]:
heart = np.array(new_data['heart'])
for i in range(len(heart)):
    print(heart[2])

83.0
